***TBMM OTURUM TUTANAKLARININ İNDİRİLMESİ***

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
from dateutil.parser import parse
import pandas as pd
import csv
import re
import time
import random

In [15]:
ilk_hedef = requests.get("https://www.tbmm.gov.tr/tutanak/tutanaklar.htm").content
soup1=BeautifulSoup(ilk_hedef)

urls_1=[]
for i in soup1.find_all("a", href=True):
    urls_1.append(str(i).split('="')[1].split('" s')[0])

# YUKARIDA ELDE EDİLEN urls_1 LİSTESİ KULLANILARAK urls_dict (TARİH:LİNK) ŞEKLİNDE ELDE EDİLİYOR
# 

urls_dict = {}

for url in urls_1:
    ikinci_hedef = requests.get(url).content
    soup2 = BeautifulSoup(ikinci_hedef)

    for i in soup2.find_all("a", href=True):
        if (not "Özet" in i) and (not "Açık Oylama Sonuçları" in i) and (not "Sesli Özet" in i) and (not "İşaret Dili" in i):             
            j = str(i).split('="')[1].split('">')[0]
            k = str(i.parent.next_sibling.next_sibling).split("d>")[1].split(" ")[0].replace(".","-")
            if ("ham" in j) or ("bas" in j):
                urls_dict[k] = j
      


    
# urls_dict SÖZLÜĞÜ YEDEKLEMEK AMACIYLA 'urls_dict.txt' DOSYASINA KAYDEDİLDİ
# with open('urls_dict.txt', 'w') as f:
#     print(urls_dict, file=f)  


urls_yedek = urls_dict.copy()         # YEDEĞİ ALMAK HER ZAMAN LAZIM

In [40]:

counter = 0
for key,value in urls_dict.items():
    with open("tamamlanan_urller.txt", 'r') as f:
        temp=list(set(f.readlines()))
        for i in range(len(temp)):
            temp[i]=temp[i].strip("\n")
        if value not in temp:
            req = requests.get(value).content
            soup = BeautifulSoup(req)
            text = soup.get_text().replace("\xa0","")
        #     date = key.strftime("%Y-%m-%d")
            with open(f"tutanaklar/{key}.txt", 'wb') as f:
                f.write(text.encode("utf-8"))

            with open("tamamlanan_urller.txt", 'a') as f:      # PROGRAM KAPANIRSA TAMAMLANMIŞ İŞLEMLER KAYBEDİLMESİN DİYE
                f.write(value)
                f.write("\r\n")

            counter+=1
            time.sleep(random.randint(10,30))                    # SİTEYİ BOĞMAMAK İÇİN
    
#     break
print(f'{counter} adet sayfa indirildi ve kaydedildi.')
    

3017 adet sayfa indirildi ve kaydedildi.
